## How To: Calculating spectral vegetation indices with EMIT data 

### Step 1: Load required Python modules

In [ ]:
import os, sys, fnmatch
import warnings
import csv
from osgeo import gdal
import numpy as np
import math
import rasterio as rio
import xarray as xr
import holoviews as hv
import hvplot.xarray
import netCDF4 as nc

sys.path.append(os.path.join(os.path.expanduser("~"),"HYR-SENSE","tools","functions"))
from spectral_index import *

# This will ignore some warnings caused by holoviews
warnings.simplefilter('ignore') 

### Step 2: Define input file path and load example data

In this example we will use the example data downloaded in the other "How To" example **"Download_data_granules_basic_example"**

In [ ]:
### Example image to use
workflow = "examples"
platform = "emit"
source_file_path = os.path.join(os.path.expanduser("~"),"HYR-SENSE","data",workflow,platform)
granules = fnmatch.filter(os.listdir(source_file_path), '*ortho.nc')
granules

In [ ]:
## Load the selected image to memory - we will use the scene ID EMIT_L2A_RFL_001_20230729T205630_2321014_019 in this example
img_file = 'EMIT_L2A_RFL_001_20230729T205630_2321014_019_ortho.nc'
img_file_dat = os.path.join(source_file_path,img_file)
ds_geo = xr.open_dataset(img_file_dat)
ds_geo

Here we calculate a simple index, the normalized difference vegetation index

In [ ]:
refl650 = ds_geo.sel(wavelengths=650, method='nearest')
refl850 = ds_geo.sel(wavelengths=850, method='nearest')
ndvi = (refl850-refl650)/(refl850+refl650)

In [ ]:
ndvi.hvplot.image(cmap='viridis', geo=True, tiles='ESRI', aspect = 'equal', frame_width=720, clim=(0,1)).opts(title="NDVI Image")

### Using a function to calculate NDVI

We can also create a custom function to calculate different normalized-difference vegetation indices, like NDVI.  We have provided one example here: 

In [ ]:
def normalized_diff_demo(input_xarray = ds_geo, band1=650, band2=850):
    """
    This function takes an input xarray image and calculates a NDVI image based on the selected bands.  The assumption is the input image is an EMIT image in 
    xarray format prepared using the emit_xarray() function from emit_tools

    The function format is Band2-Band1 / Band2+Band1

    param: input_xarray is the input EMIT xarray in memory
    param: band1 the EMIT band number to use for band 1
    param: band3 the EMIT band number to use for band 3

    
    """
    
    reflb1 = input_xarray.sel(wavelengths=band1, method='nearest')
    reflb2 = input_xarray.sel(wavelengths=band2, method='nearest')
    ndiff_image = (reflb2-reflb1)/(reflb2+reflb1)
    
    return(ndiff_image)

In [ ]:
ndvi2 = normalized_diff_demo(input_xarray = ds_geo, band1=650, band2=850)

In [ ]:
ndvi2.hvplot.image(cmap='viridis', geo=True, tiles='ESRI', aspect = 'equal', frame_width=720, clim=(0,1)).opts(title="NDVI Image")

We can also use the stored functions in the HYR-SENSE repository

In [ ]:
ndvi3 = normalized_diff(input_xarray = ds_geo, band1=650, band2=850)

In [ ]:
ndvi3.hvplot.image(cmap='viridis', geo=True, tiles='ESRI', aspect = 'equal', frame_width=720, clim=(0,1)).opts(title="NDVI Image")

### Calculate the Normalized-Difference Water Index (NDWI)

https://en.wikipedia.org/wiki/Normalized_difference_water_index

In [ ]:
ds_geo['wavelengths'].data

In [ ]:
ndwi = normalized_diff(input_xarray = ds_geo, band1=2200, band2=864)

In [ ]:
ndwi.hvplot.image(cmap='viridis', geo=True, tiles='ESRI', aspect = 'equal', frame_width=720, clim=(0,1)).opts(title="NDVI Image")